In [1]:
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [2]:
import tensorflow as tf

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [4]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [5]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [6]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        if activation=="relu":
            return tf.nn.relu(z)
        else:
            return z

In [7]:
#with tf.name_scope("dnn"):
#    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="relu")
#    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
#    logits = neuron_layer(hidden2, n_outputs, "outputs")

In [8]:
from tensorflow.contrib.layers import fully_connected

with tf.name_scope("dnn"):
    hidden1 = fully_connected(X, n_hidden1, scope="hidden1")
    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2")
    logits = fully_connected(hidden2, n_outputs, scope="outputs", activation_fn=None)

In [9]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [10]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [11]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [12]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [13]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

W0627 16:02:03.578927 140187694044992 deprecation.py:323] From <ipython-input-13-c50d5bb4a85c>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0627 16:02:03.579595 140187694044992 deprecation.py:323] From /home/sean/venv/py37/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0627 16:02:03.580311 140187694044992 deprecation.py:323] From /home/sean/venv/py37/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz


W0627 16:02:03.782274 140187694044992 deprecation.py:323] From /home/sean/venv/py37/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [14]:
n_epochs = 400
batch_size = 50

In [15]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    save_path = saver.save(sess, "./dnn_mnist_final.ckpt")

0 Train accuracy: 0.92 Test accuracy: 0.9068
1 Train accuracy: 0.9 Test accuracy: 0.9214
2 Train accuracy: 0.96 Test accuracy: 0.9317
3 Train accuracy: 1.0 Test accuracy: 0.939
4 Train accuracy: 0.96 Test accuracy: 0.9427
5 Train accuracy: 0.9 Test accuracy: 0.9466
6 Train accuracy: 0.96 Test accuracy: 0.9495
7 Train accuracy: 0.98 Test accuracy: 0.9511
8 Train accuracy: 0.96 Test accuracy: 0.9545
9 Train accuracy: 0.98 Test accuracy: 0.9582
10 Train accuracy: 0.96 Test accuracy: 0.9603
11 Train accuracy: 0.96 Test accuracy: 0.9616
12 Train accuracy: 1.0 Test accuracy: 0.9627
13 Train accuracy: 0.98 Test accuracy: 0.9641
14 Train accuracy: 0.98 Test accuracy: 0.966
15 Train accuracy: 0.98 Test accuracy: 0.9673
16 Train accuracy: 1.0 Test accuracy: 0.9693
17 Train accuracy: 0.98 Test accuracy: 0.9689
18 Train accuracy: 0.96 Test accuracy: 0.9693
19 Train accuracy: 1.0 Test accuracy: 0.9712
20 Train accuracy: 0.98 Test accuracy: 0.9719
21 Train accuracy: 0.98 Test accuracy: 0.972
22 Trai

181 Train accuracy: 1.0 Test accuracy: 0.9801
182 Train accuracy: 1.0 Test accuracy: 0.9805
183 Train accuracy: 1.0 Test accuracy: 0.9803
184 Train accuracy: 1.0 Test accuracy: 0.9803
185 Train accuracy: 1.0 Test accuracy: 0.9804
186 Train accuracy: 1.0 Test accuracy: 0.9808
187 Train accuracy: 1.0 Test accuracy: 0.9803
188 Train accuracy: 1.0 Test accuracy: 0.9802
189 Train accuracy: 1.0 Test accuracy: 0.98
190 Train accuracy: 1.0 Test accuracy: 0.9807
191 Train accuracy: 1.0 Test accuracy: 0.9804
192 Train accuracy: 1.0 Test accuracy: 0.9805
193 Train accuracy: 1.0 Test accuracy: 0.9801
194 Train accuracy: 1.0 Test accuracy: 0.9803
195 Train accuracy: 1.0 Test accuracy: 0.9807
196 Train accuracy: 1.0 Test accuracy: 0.9805
197 Train accuracy: 1.0 Test accuracy: 0.9804
198 Train accuracy: 1.0 Test accuracy: 0.9806
199 Train accuracy: 1.0 Test accuracy: 0.9805
200 Train accuracy: 1.0 Test accuracy: 0.9805
201 Train accuracy: 1.0 Test accuracy: 0.9804
202 Train accuracy: 1.0 Test accurac

360 Train accuracy: 1.0 Test accuracy: 0.9803
361 Train accuracy: 1.0 Test accuracy: 0.9803
362 Train accuracy: 1.0 Test accuracy: 0.9803
363 Train accuracy: 1.0 Test accuracy: 0.9804
364 Train accuracy: 1.0 Test accuracy: 0.9803
365 Train accuracy: 1.0 Test accuracy: 0.9805
366 Train accuracy: 1.0 Test accuracy: 0.9802
367 Train accuracy: 1.0 Test accuracy: 0.9806
368 Train accuracy: 1.0 Test accuracy: 0.9806
369 Train accuracy: 1.0 Test accuracy: 0.9805
370 Train accuracy: 1.0 Test accuracy: 0.9803
371 Train accuracy: 1.0 Test accuracy: 0.9806
372 Train accuracy: 1.0 Test accuracy: 0.9803
373 Train accuracy: 1.0 Test accuracy: 0.9803
374 Train accuracy: 1.0 Test accuracy: 0.9804
375 Train accuracy: 1.0 Test accuracy: 0.9804
376 Train accuracy: 1.0 Test accuracy: 0.9805
377 Train accuracy: 1.0 Test accuracy: 0.9803
378 Train accuracy: 1.0 Test accuracy: 0.9802
379 Train accuracy: 1.0 Test accuracy: 0.9803
380 Train accuracy: 1.0 Test accuracy: 0.9805
381 Train accuracy: 1.0 Test accur

In [16]:
with tf.Session() as sess:
    saver.restore(sess, "./dnn_mnist_final.ckpt")
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

W0627 16:12:33.873679 140187694044992 deprecation.py:323] From /home/sean/venv/py37/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [17]:
y_pred

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4])

In [18]:
y_test[:20]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4],
      dtype=int32)